In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

def calculate_2year_beta():
    # Stock tickers for reference
    tickers = ['META', 'REGN', 'FI', 'GILD', 'MITK', 'FIS', 'IDCC', 'COUR', 'AMAT', 'KLAC',
               'NICE', 'WFC', 'AMD', 'LLY', 'NVO', 'DELL', 'BP', 'NOK', 'RMBS', 'EL',
               'UIS', 'ILMN', 'GOOG', 'SNPS', 'GRAL', 'UNH', 'GNRC', 'TXN', 'ROK']

    print("Fetching 2-year historical data from Yahoo Finance...")

    # Define date ranges for quarters
    end_date = datetime(2025, 6, 30)  # End of Q2 2025
    start_date = datetime(2023, 1, 1)  # Start of Q1 2023

    # Fetch S&P 500 data
    sp500 = yf.download('^GSPC', start=start_date, end=end_date, progress=False, auto_adjust=False)

    # Check column structure and handle MultiIndex
    if len(sp500.columns.levels) > 1:
        # MultiIndex columns
        sp500_close = sp500[('Adj Close', '^GSPC')]
    else:
        # Single level columns
        sp500_close = sp500['Adj Close']

    # Calculate quarterly returns for S&P 500
    sp500_quarterly = sp500_close.resample('Q').last()
    sp500_returns = sp500_quarterly.pct_change().dropna() * 100

    print(f"S&P 500 quarterly returns: {sp500_returns.values}")

    # Fetch stock data and calculate betas
    betas_2y = []
    valid_tickers = []

    for ticker in tickers:
        try:
            # Fetch stock data
            stock = yf.download(ticker, start=start_date, end=end_date, progress=False, auto_adjust=False)

            if not stock.empty:
                # Handle column structure
                if len(stock.columns.levels) > 1:
                    stock_close = stock[('Adj Close', ticker)]
                else:
                    stock_close = stock['Adj Close']

                # Calculate quarterly returns
                stock_quarterly = stock_close.resample('Q').last()
                stock_returns = stock_quarterly.pct_change().dropna() * 100

                # Align dates with S&P 500
                aligned_dates = stock_returns.index.intersection(sp500_returns.index)

                if len(aligned_dates) >= 4:  # Need at least 4 quarters
                    stock_aligned = stock_returns[aligned_dates]
                    market_aligned = sp500_returns[aligned_dates]

                    # Calculate beta
                    covariance = np.cov(stock_aligned, market_aligned)[0, 1]
                    market_variance = np.var(market_aligned, ddof=1)

                    if market_variance != 0:
                        beta = covariance / market_variance
                    else:
                        beta = 0

                    betas_2y.append(beta)
                    valid_tickers.append(ticker)
                    print(f"{ticker}: Beta = {beta:.4f}")
                else:
                    print(f"{ticker}: Insufficient data")
            else:
                print(f"{ticker}: No data found")

        except Exception as e:
            print(f"{ticker}: Error fetching data - {str(e)}")

    # Create DataFrame with valid data
    df_2y_original = pd.DataFrame({
        'Stock': valid_tickers,
        'Beta_2Y': betas_2y
    })

    # Create sorted DataFrame
    df_2y_sorted = df_2y_original.sort_values('Beta_2Y', ascending=False)

    # Save to CSV files
    df_2y_original.to_csv('beta_2year_original_order.csv', index=False)
    df_2y_sorted.to_csv('beta_2year_sorted_by_value.csv', index=False)

    print(f"\nProcessed {len(valid_tickers)} stocks successfully")

    return df_2y_sorted, df_2y_original

if __name__ == "__main__":
    print("2-Year Beta Analysis")
    print("=" * 50)

    beta_2y_sorted, beta_2y_original = calculate_2year_beta()

    print("\n2-Year Beta Analysis (Sorted by Beta):")
    print(beta_2y_sorted.to_string(index=False, float_format='%.4f'))

    print("\n2-Year Beta Analysis (Original Order):")
    print(beta_2y_original.to_string(index=False, float_format='%.4f'))



2-Year Beta Analysis
Fetching 2-year historical data from Yahoo Finance...
S&P 500 quarterly returns: [ 8.29992917 -3.64755554 11.23541692 10.15801426  3.9230329   5.53064934
  2.06768445 -4.58682016 10.00061863]


C:\Users\Owner\AppData\Local\Temp\ipykernel_35224\4091619398.py:30: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  sp500_quarterly = sp500_close.resample('Q').last()
C:\Users\Owner\AppData\Local\Temp\ipykernel_35224\4091619398.py:52: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  stock_quarterly = stock_close.resample('Q').last()
C:\Users\Owner\AppData\Local\Temp\ipykernel_35224\4091619398.py:52: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  stock_quarterly = stock_close.resample('Q').last()


META: Beta = 2.0167
REGN: Beta = 0.0605


C:\Users\Owner\AppData\Local\Temp\ipykernel_35224\4091619398.py:52: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  stock_quarterly = stock_close.resample('Q').last()
C:\Users\Owner\AppData\Local\Temp\ipykernel_35224\4091619398.py:52: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  stock_quarterly = stock_close.resample('Q').last()


FI: Beta = 0.5381
GILD: Beta = -0.7663


...

C:\Users\Owner\AppData\Local\Temp\ipykernel_35224\4091619398.py:52: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  stock_quarterly = stock_close.resample('Q').last()


GNRC: Beta = 2.6023


C:\Users\Owner\AppData\Local\Temp\ipykernel_35224\4091619398.py:52: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  stock_quarterly = stock_close.resample('Q').last()


TXN: Beta = 1.0388
ROK: Beta = 1.4377

Processed 29 stocks successfully

2-Year Beta Analysis (Sorted by Beta):
Stock  Beta_2Y
  AMD   3.4929
 GNRC   2.6023
  UIS   2.5803
 ILMN   2.4202
 AMAT   2.2465
 DELL   2.1442
 META   2.0167
 NICE   1.9827
 KLAC   1.8824
 MITK   1.8234
 GRAL   1.8211
   EL   1.8104
  ROK   1.4377
 IDCC   1.3868
 RMBS   1.2584
  NVO   1.2072
 SNPS   1.1977
  FIS   1.1093
  TXN   1.0388
  WFC   1.0162
 GOOG   0.8971
  LLY   0.5910
   FI   0.5381
 REGN   0.0605
 COUR  -0.1751
  NOK  -0.6624
 GILD  -0.7663
  UNH  -0.8711
   BP  -1.1141

2-Year Beta Analysis (Original Order):
Stock  Beta_2Y
 META   2.0167
 REGN   0.0605
   FI   0.5381
 GILD  -0.7663
 MITK   1.8234
  FIS   1.1093
 IDCC   1.3868
 COUR  -0.1751
 AMAT   2.2465
 KLAC   1.8824
 NICE   1.9827
  WFC   1.0162
  AMD   3.4929
  LLY   0.5910
  NVO   1.2072
 DELL   2.1442
   BP  -1.1141
  NOK  -0.6624
 RMBS   1.2584
   EL   1.8104
  UIS   2.5803
 ILMN   2.4202
 GOOG   0.8971
 SNPS   1.1977
 GRAL   1.8211
  UNH  -

C:\Users\Owner\AppData\Local\Temp\ipykernel_35224\4091619398.py:52: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  stock_quarterly = stock_close.resample('Q').last()
